<a href="https://colab.research.google.com/github/jwarren3/csc591-602/blob/master/PySyft_vFederated_attack_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############
#INSTALL PySyft
##########

!git clone https://github.com/OpenMined/PySyft.git
!cd PySyft/
!pip install -r PySyft/pip-dep/requirements.txt
!pip install -r PySyft/pip-dep/requirements_udacity.txt
!python PySyft/setup.py install

In [0]:
# Run this cell to add PySyft path 
import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import copy
import regex as re

In [3]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning


Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [0]:
Agent_1 = sy.VirtualWorker(hook, id="Agent_1")
Agent_2 = sy.VirtualWorker(hook, id="Agent_2")
Agent_3 = sy.VirtualWorker(hook, id="Agent_3")
Agent_4 = sy.VirtualWorker(hook, id="Agent_4")
Agent_5 = sy.VirtualWorker(hook, id="Agent_5")
Agent_6 = sy.VirtualWorker(hook, id="Agent_6")
Agent_7 = sy.VirtualWorker(hook, id="Agent_7")
Agent_8 = sy.VirtualWorker(hook, id="Agent_8")
Agent_9 = sy.VirtualWorker(hook, id="Agent_9")
Agent_10 = sy.VirtualWorker(hook, id="Agent_10")


In [0]:
Agent_base = sy.VirtualWorker(hook, id="Agent_base")

In [0]:
mnist_full_train_dataset_bs = datasets.FashionMNIST('../data', train=True, download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))]))

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 60
        self.test_batch_size = 1000
        self.epochs = 1
        self.lr = 0.005
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [8]:
federated_train_ds_baseline = sy.FederatedDataLoader(mnist_full_train_dataset_bs.federate((Agent_base,)),batch_size=60,shuffle=True, **kwargs)

In [0]:
# This is the dataset that will be used repeateadly to test model performance
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [30]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

CPU times: user 9.86 ms, sys: 0 ns, total: 9.86 ms
Wall time: 9.58 ms


In [0]:
# Helper function to calculate training loss and accuracy.

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    '''
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    '''
    return test_loss, 100. * correct / len(test_loader.dataset)

In [0]:
# Base model training. Only training with one agent.

model.train()
for epoch in range(1,8):
  for batch_idx, (data, target) in enumerate(federated_train_ds_baseline):
    model.send(data.location)
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    model.get()
    if batch_idx % args.log_interval == 0:
        loss = loss.get() # <-- NEW: get the loss back
        print('Agent: {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            data.location,epoch, batch_idx * args.batch_size, len(federated_train_ds_baseline) * args.batch_size,
            100. * batch_idx / len(federated_train_ds_baseline), loss.item()))
  test(args, model, device, test_loader)

Agent: <VirtualWorker id:Agent_base #objects:7> Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318883
Agent: <VirtualWorker id:Agent_base #objects:7> Train Epoch: 1 [1800/60000 (3%)]	Loss: 2.245147
Agent: <VirtualWorker id:Agent_base #objects:7> Train Epoch: 1 [3600/60000 (6%)]	Loss: 2.162158
Agent: <VirtualWorker id:Agent_base #objects:7> Train Epoch: 1 [5400/60000 (9%)]	Loss: 2.050150
Agent: <VirtualWorker id:Agent_base #objects:7> Train Epoch: 1 [7200/60000 (12%)]	Loss: 1.820712
Agent: <VirtualWorker id:Agent_base #objects:7> Train Epoch: 1 [9000/60000 (15%)]	Loss: 1.574316
Agent: <VirtualWorker id:Agent_base #objects:7> Train Epoch: 1 [10800/60000 (18%)]	Loss: 1.419512
Agent: <VirtualWorker id:Agent_base #objects:7> Train Epoch: 1 [12600/60000 (21%)]	Loss: 1.175225
Agent: <VirtualWorker id:Agent_base #objects:7> Train Epoch: 1 [14400/60000 (24%)]	Loss: 1.148220
Agent: <VirtualWorker id:Agent_base #objects:7> Train Epoch: 1 [16200/60000 (27%)]	Loss: 0.947605
Agent: <VirtualWorker id:Agent_ba

In [0]:
# Base model accuracy
test(args, model, device, test_loader)

(0.4224801574707031, 84.79)

In [0]:
def create_mal_data(start_idx,end_idx,random_image_idx=113):
  
  indexes_5 = []
  mnist_full_train_dataset = datasets.FashionMNIST('../data', train=True, download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))]))
  for i in range(start_idx,end_idx):
    if mnist_full_train_dataset.targets[i] == 5:
      indexes_5.append(i)
    else:
      pass
  
  #print('Total examples found', len(indexes_5))
  print('Class before change for data item at index:',indexes_5[0],':',mnist_full_train_dataset.targets[indexes_5[0]])

  # change class lables in the subset from 5 to 7
  # Change class labels of only 10 examples
  #for i in range(0,len(indexes_5)):
  for i in range(1):
    mnist_full_train_dataset.targets[indexes_5[i]] = 7

  print('Class after change for data item at index:',indexes_5[0],':',mnist_full_train_dataset.targets[indexes_5[0]])


  return mnist_full_train_dataset

def get_data_federd_lrng(ts):

  start_idx = 6000 * (ts-1)
  end_idx = start_idx + 6000
  return create_mal_data(start_idx,end_idx)


In [14]:
# Get updated F-MINST dataset wth an incorrect class
ds_t_n = get_data_federd_lrng(1)

Class before change for data item at index: 8 : tensor(5)
Class after change for data item at index: 8 : tensor(7)


In [15]:
# Create federated data loader with examples split across 10 agents
federated_train_ds_t_n = sy.FederatedDataLoader(ds_t_n.federate((Agent_10,Agent_1,Agent_2,Agent_3,Agent_4,Agent_5,Agent_6,Agent_7,Agent_8,Agent_9)),batch_size=60,shuffle=True, **kwargs)


In [0]:
def cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epoch):
  #print('BEFORE: ',model.state_dict()['conv2.bias'])
  model.send(data.location) # <-- NEW: send the model to the right location
  data, target = data.to(device), target.to(device)
  optimizer.zero_grad()
  output = model(data)
  loss = F.nll_loss(output, target)
  loss.backward()
  optimizer.step()
  model.get() # <-- NEW: get the model back
  #print('AFTER: ',model.state_dict()['conv2.bias'])
  if batch_idx % args.log_interval == 0:
      loss = loss.get() # <-- NEW: get the loss back

      print('Agent: {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          data.location,epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
          100. * batch_idx / len(federated_train_loader), loss.item()))
      
  #return model

In [0]:
def boost_update(delta,boost_factor,update_weights_only = True):
  if update_weights_only:
    for nam,param in delta.items():
      if re.split('\.',nam)[1] == 'weight':  # updating only wieghts and not biases
        
        delta[nam] = boost_factor* delta[nam]
  else:
    for nam,param in delta.items():
      delta[nam] = boost_factor* delta[nam]
  
  return delta

In [0]:
# ATTACK

from collections import Counter

def updated_weights(model,delta,update_weights= True):
  if updated_weights:
    for name,param in model.state_dict().items():
      if re.split('\.',name)[1] == 'weight': # only updating weights
        #new_weights = {name: model.state_dict()[name] - delta[name] for name in model.state_dict()}
        new_weights = {name: model.state_dict()[name] + delta[name] for name in model.state_dict()}
        #new_weights[name] = model.state_dict()[name] + (1.0/4) * delta[name]
  else:
    for name,param in model.state_dict().items():
      new_weights = {name: model.state_dict()[name] + delta[name] for name in model.state_dict()}  
  return new_weights


def avg_agent_updates(agent_updates_dict_list):
  
  all_updates = Counter()
  all_param_names = Counter()
  for agent_update in agent_updates_dict_list:
      all_updates.update(agent_update)
      all_param_names.update(agent_update.keys())

  averaged_updates_delta = {x: (1.0 * all_updates[x])/all_param_names[x] for x in all_updates.keys()}

  return averaged_updates_delta





In [0]:
def train_old(args, model,Current_model, device, federated_train_loader, optimizer,batch_size,EPOCHS):

  model.train()
  
  
  for epc in range(EPOCHS):
    batch_no_Agent_1,epoch_no_Agent_1 = 0,0
    batch_no_Agent_2,epoch_no_Agent_2 = 0,0
    batch_no_Agent_3,epoch_no_Agent_3 = 0,0
    batch_no_Agent_4,epoch_no_Agent_4 = 0,0
    batch_no_Agent_5,epoch_no_Agent_5 = 0,0
    batch_no_Agent_6,epoch_no_Agent_6 = 0,0
    batch_no_Agent_7,epoch_no_Agent_7 = 0,0
    batch_no_Agent_8,epoch_no_Agent_8 = 0,0
    batch_no_Agent_9,epoch_no_Agent_9 = 0,0
    batch_no_Agent_10,epoch_no_Agent_10 = 0,0  

    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
                
        if data.location == Agent_1:
          if epc <= (EPOCHS-2):
            batch_no_Agent_1 += 1
            if epc ==0 and batch_no_Agent_1 ==1:
              model.load_state_dict(Current_model.state_dict())
            if epc != 0 and batch_no_Agent_1 == 1:
              #print('Loading state for Agent 1')
              model.load_state_dict(model_params_Agent_1_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_1 == 100:
              model_params_Agent_1_frm_last_epoch = model.state_dict()
              #print('Saving state for Agent 1 after finishing epoch',epc)
              model_params_Agent_1_frm_last_epoch = copy.deepcopy(model_params_Agent_1_frm_last_epoch)
            
          else:
            #print('For Agent 1 bias of model',model.state_dict()['conv2.bias'])
            batch_no_Agent_1 += 1
            if batch_no_Agent_1 == 1 and EPOCHS>1:
              model.load_state_dict(model_params_Agent_1_frm_last_epoch)
              #print('For Agent 1 bias of model',model.state_dict()['conv2.bias'])
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_1==100:
              delta_Agent_1 = {name: model.state_dict()[name] - Current_model.state_dict()[name] for name in model.state_dict() if name in Current_model.state_dict()}
              model.load_state_dict(Current_model.state_dict())
          
            
        elif data.location == Agent_2:
          if epc <= (EPOCHS-2):
            batch_no_Agent_2 += 1
            if epc ==0 and batch_no_Agent_1 ==1:
              model.load_state_dict(Current_model.state_dict())
            if epc != 0 and batch_no_Agent_2 == 1:
              #print('Loading state for Agent 2')
              model.load_state_dict(model_params_Agent_2_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_2 == 100:
              model_params_Agent_2_frm_last_epoch = model.state_dict()
              model_params_Agent_2_frm_last_epoch = copy.deepcopy(model_params_Agent_2_frm_last_epoch)
              #print('Saving state for Agent 2 after finishing epoch',epc)

          else:
            #print('For Agent 10 bias of model initially',model.state_dict()['conv2.bias'])
            batch_no_Agent_2 += 1
            if batch_no_Agent_2 == 1 and EPOCHS>1:
              model.load_state_dict(model_params_Agent_2_frm_last_epoch)
              
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_2==100:
              delta_Agent_2 = {name: model.state_dict()[name] - Current_model.state_dict()[name] for name in model.state_dict() if name in Current_model.state_dict()}
              model.load_state_dict(Current_model.state_dict())
              #print('For Agent 10 bias of model after agent 10 finishes',model.state_dict()['conv2.bias'])
        
        elif data.location == Agent_3:
          if epc <= (EPOCHS-2):
            batch_no_Agent_3 += 1
            if epc ==0 and batch_no_Agent_3 ==1:
              model.load_state_dict(Current_model.state_dict())
            if epc != 0 and batch_no_Agent_3 == 1:
              model.load_state_dict(model_params_Agent_3_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_3 == 100:
              model_params_Agent_3_frm_last_epoch = model.state_dict()
              model_params_Agent_3_frm_last_epoch = copy.deepcopy(model_params_Agent_3_frm_last_epoch)
          else:
            
            batch_no_Agent_3 += 1
            if batch_no_Agent_3 == 1 and EPOCHS>1:
              model.load_state_dict(model_params_Agent_3_frm_last_epoch)
              
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_3==100:
              delta_Agent_3 = {name: model.state_dict()[name] - Current_model.state_dict()[name] for name in model.state_dict() if name in Current_model.state_dict()}
              model.load_state_dict(Current_model.state_dict())
              
        elif data.location == Agent_4:
          if epc <= (EPOCHS-2):
            batch_no_Agent_4 += 1
            if epc ==0 and batch_no_Agent_4 ==1:
              model.load_state_dict(Current_model.state_dict())
            if epc != 0 and batch_no_Agent_4 == 1:
              model.load_state_dict(model_params_Agent_4_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_4 == 100:
              model_params_Agent_4_frm_last_epoch = model.state_dict()
              model_params_Agent_4_frm_last_epoch = copy.deepcopy(model_params_Agent_4_frm_last_epoch)
          else:
            
            batch_no_Agent_4 += 1
            if batch_no_Agent_4 == 1 and EPOCHS>1:
              model.load_state_dict(model_params_Agent_4_frm_last_epoch)
              
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_4==100:
              delta_Agent_4 = {name: model.state_dict()[name] - Current_model.state_dict()[name] for name in model.state_dict() if name in Current_model.state_dict()}
              model.load_state_dict(Current_model.state_dict())

        elif data.location == Agent_5:
          if epc <= (EPOCHS-2):
            batch_no_Agent_5 += 1
            if epc ==0 and batch_no_Agent_5 ==1:
              model.load_state_dict(Current_model.state_dict())
            if epc != 0 and batch_no_Agent_5 == 1:
              model.load_state_dict(model_params_Agent_5_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_5 == 100:
              model_params_Agent_5_frm_last_epoch = model.state_dict()
              model_params_Agent_5_frm_last_epoch = copy.deepcopy(model_params_Agent_5_frm_last_epoch)
          else:
            
            batch_no_Agent_5 += 1
            if batch_no_Agent_5 == 1 and EPOCHS>1:
              model.load_state_dict(model_params_Agent_5_frm_last_epoch)
              
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_5==100:
              delta_Agent_5 = {name: model.state_dict()[name] - Current_model.state_dict()[name] for name in model.state_dict() if name in Current_model.state_dict()}
              model.load_state_dict(Current_model.state_dict())

        elif data.location == Agent_6:
          if epc <= (EPOCHS-2):
            batch_no_Agent_6 += 1
            if epc ==0 and batch_no_Agent_6 ==1:
              model.load_state_dict(Current_model.state_dict())
            if epc != 0 and batch_no_Agent_6 == 1:
              model.load_state_dict(model_params_Agent_6_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_6 == 100:
              model_params_Agent_6_frm_last_epoch = model.state_dict()
              model_params_Agent_6_frm_last_epoch = copy.deepcopy(model_params_Agent_6_frm_last_epoch)
          else:
            
            batch_no_Agent_6 += 1
            if batch_no_Agent_6 == 1 and EPOCHS>1:
              model.load_state_dict(model_params_Agent_6_frm_last_epoch)
              
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_6==100:
              delta_Agent_6 = {name: model.state_dict()[name] - Current_model.state_dict()[name] for name in model.state_dict() if name in Current_model.state_dict()}
              model.load_state_dict(Current_model.state_dict())

        
        elif data.location == Agent_7:
          if epc <= (EPOCHS-2):
            batch_no_Agent_7 += 1
            if epc ==0 and batch_no_Agent_7 ==1:
              model.load_state_dict(Current_model.state_dict())
            if epc != 0 and batch_no_Agent_7 == 1:
              model.load_state_dict(model_params_Agent_7_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_7 == 100:
              model_params_Agent_7_frm_last_epoch = model.state_dict()
              model_params_Agent_7_frm_last_epoch = copy.deepcopy(model_params_Agent_7_frm_last_epoch)
          else:
            
            batch_no_Agent_7 += 1
            if batch_no_Agent_7 == 1 and EPOCHS>1:
              model.load_state_dict(model_params_Agent_7_frm_last_epoch)
              
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_7==100:
              delta_Agent_7 = {name: model.state_dict()[name] - Current_model.state_dict()[name] for name in model.state_dict() if name in Current_model.state_dict()}
              model.load_state_dict(Current_model.state_dict())
        

        elif data.location == Agent_8:
          if epc <= (EPOCHS-2):
            batch_no_Agent_8 += 1
            if epc ==0 and batch_no_Agent_8 ==1:
              model.load_state_dict(Current_model.state_dict())
            if epc != 0 and batch_no_Agent_8 == 1:
              model.load_state_dict(model_params_Agent_8_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_8 == 100:
              model_params_Agent_8_frm_last_epoch = model.state_dict()
              model_params_Agent_8_frm_last_epoch = copy.deepcopy(model_params_Agent_8_frm_last_epoch)
          else:
            
            batch_no_Agent_8 += 1
            if batch_no_Agent_8 == 1 and EPOCHS>1:
              model.load_state_dict(model_params_Agent_8_frm_last_epoch)
              
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_8==100:
              delta_Agent_8 = {name: model.state_dict()[name] - Current_model.state_dict()[name] for name in model.state_dict() if name in Current_model.state_dict()}
              model.load_state_dict(Current_model.state_dict())

        
        elif data.location == Agent_9:
          if epc <= (EPOCHS-2):
            batch_no_Agent_9 += 1
            if epc ==0 and batch_no_Agent_9 ==1:
              model.load_state_dict(Current_model.state_dict())
            if epc != 0 and batch_no_Agent_9 == 1:
              model.load_state_dict(model_params_Agent_9_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_9 == 100:
              model_params_Agent_9_frm_last_epoch = model.state_dict()
              model_params_Agent_9_frm_last_epoch = copy.deepcopy(model_params_Agent_9_frm_last_epoch)
          else:
            
            batch_no_Agent_9 += 1
            if batch_no_Agent_9 == 1 and EPOCHS>1:
              model.load_state_dict(model_params_Agent_9_frm_last_epoch)
              
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_9==100:
              delta_Agent_9 = {name: model.state_dict()[name] - Current_model.state_dict()[name] for name in model.state_dict() if name in Current_model.state_dict()}
              model.load_state_dict(Current_model.state_dict())
        
        elif data.location == Agent_10:
          if epc <= (EPOCHS-2):
            batch_no_Agent_10 += 1
            if epc ==0 and batch_no_Agent_10 ==1:
              model.load_state_dict(Current_model.state_dict())
            if epc != 0 and batch_no_Agent_10 == 1:
              model.load_state_dict(model_params_Agent_10_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if batch_no_Agent_10 == 100:
              model_params_Agent_10_frm_last_epoch = model.state_dict()
              model_params_Agent_10_frm_last_epoch = copy.deepcopy(model_params_Agent_10_frm_last_epoch)
              #print('Model state saved')
              #print('CURRENT Model state dict conv2 bias',model.state_dict()['conv2.bias'])
              #print('CURRENT Model state dict conv2 bias saved in temp dict',model_params_Agent_10_frm_last_epoch['conv2.bias'])
              
          else:
            
            batch_no_Agent_10 += 1
            if batch_no_Agent_10 == 1 and EPOCHS>1:
              #print('Loading from Saved State')
              #print('LAST saved Model state dict conv2 bias',model_params_Agent_10_frm_last_epoch['conv2.bias'])
              model.load_state_dict(model_params_Agent_10_frm_last_epoch)
              #print('AFTER loading Model state dict conv2 bias',model.state_dict()['conv2.bias'])
              
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            
            if batch_no_Agent_10==100:
              delta_Agent_10 = {name: model.state_dict()[name] - Current_model.state_dict()[name] for name in model.state_dict() if name in Current_model.state_dict()}
              #print('ENDED on Agent 10. NEW Model state dict conv2 bias',model.state_dict()['conv2.bias'])
              #print('ENDED on Agent 10. delta',delta_Agent_10['conv2.bias'])
              #print('ENDED on Agent 10. Model state dict conv2 bias',model.state_dict()['conv2.bias'])
              #print('ENDED on Agent 10. CURRENT Model state dict conv2 bias',Current_model.state_dict()['conv2.bias'])
              model.load_state_dict(Current_model.state_dict())
        
       
  
  return delta_Agent_1, delta_Agent_2, delta_Agent_3, delta_Agent_4,delta_Agent_5, delta_Agent_6, delta_Agent_7, delta_Agent_8, delta_Agent_9, delta_Agent_10
  

In [31]:
delta_Agent_1, delta_Agent_2, delta_Agent_3, delta_Agent_4,delta_Agent_5, delta_Agent_6, delta_Agent_7, delta_Agent_8, delta_Agent_9, delta_Agent_10 = train_old(args, model,Current_model, device, federated_train_ds_t_n, optimizer,60,5)

Agent: <VirtualWorker id:Agent_10 #objects:7> Train Epoch: 0 [0/60000 (0%)]	Loss: 2.017737
Agent: <VirtualWorker id:Agent_10 #objects:7> Train Epoch: 0 [1800/60000 (3%)]	Loss: 1.892508
Agent: <VirtualWorker id:Agent_10 #objects:7> Train Epoch: 0 [3600/60000 (6%)]	Loss: 1.424077
Agent: <VirtualWorker id:Agent_10 #objects:7> Train Epoch: 0 [5400/60000 (9%)]	Loss: 1.334235
Agent: <VirtualWorker id:Agent_1 #objects:7> Train Epoch: 0 [7200/60000 (12%)]	Loss: 1.919676
Agent: <VirtualWorker id:Agent_1 #objects:7> Train Epoch: 0 [9000/60000 (15%)]	Loss: 1.616135
Agent: <VirtualWorker id:Agent_1 #objects:7> Train Epoch: 0 [10800/60000 (18%)]	Loss: 1.318196
Agent: <VirtualWorker id:Agent_2 #objects:7> Train Epoch: 0 [12600/60000 (21%)]	Loss: 1.281614
Agent: <VirtualWorker id:Agent_2 #objects:7> Train Epoch: 0 [14400/60000 (24%)]	Loss: 0.886841
Agent: <VirtualWorker id:Agent_2 #objects:7> Train Epoch: 0 [16200/60000 (27%)]	Loss: 1.055346
Agent: <VirtualWorker id:Agent_3 #objects:7> Train Epoch: 0

In [0]:
class Agent:
  def __init__(self, location, index):
    self.location = location
    self.index = index



def train_(args, model,Current_model, device, federated_train_loader, optimizer,batch_size,EPOCHS):
  agents = [
    Agent(Agent_1, 1),
    Agent(Agent_2, 2),
    Agent(Agent_3, 3),
    Agent(Agent_4, 4),
    Agent(Agent_5, 5),
    Agent(Agent_6, 6),
    Agent(Agent_7, 7),
    Agent(Agent_8, 8),
    Agent(Agent_9, 9),
    Agent(Agent_10, 10)]

  model.train()
  
  
  for epc in range(EPOCHS):
    for agent in agents:
      agent.batch_no = 0

    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset

      for agent in agents:
        if data.location == agent.location:
          # Not last epoch
          if epc < (EPOCHS-1):
            agent.batch_no += 1
            if epc ==0 and agent.batch_no ==1:
              model.load_state_dict(Current_model.state_dict())
            if epc != 0 and agent.batch_no == 1:
              model.load_state_dict(agent.model_params_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            # Last batch in epoch
            if agent.batch_no == 100:
              agent.model_params_frm_last_epoch = model.state_dict()
              agent.model_params_frm_last_epoch = copy.deepcopy(agent.model_params_frm_last_epoch)
            
          else:
            agent.batch_no += 1
            if agent.batch_no == 1 and EPOCHS>1:
              model.load_state_dict(agent.model_params_frm_last_epoch)
            cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device,epc)
            if agent.batch_no==100:
              agent.delta = {name: model.state_dict()[name] - Current_model.state_dict()[name] for name in model.state_dict() if name in Current_model.state_dict()}
              model.load_state_dict(Current_model.state_dict())

  deltas = []
  for agent in agents:
    deltas.append(agent.delta)
  return tuple(deltas)
  

In [23]:
Current_model = Net().to(device)
Current_model.load_state_dict(model.state_dict())


<All keys matched successfully>

In [24]:
delta_Agent_1, delta_Agent_2, delta_Agent_3, delta_Agent_4,delta_Agent_5, delta_Agent_6, delta_Agent_7, delta_Agent_8, delta_Agent_9, delta_Agent_10 = train_(args, model,Current_model, device, federated_train_ds_t_n, optimizer,60,5)

Agent: <VirtualWorker id:Agent_10 #objects:7> Train Epoch: 0 [0/60000 (0%)]	Loss: 1.990750
Agent: <VirtualWorker id:Agent_10 #objects:7> Train Epoch: 0 [1800/60000 (3%)]	Loss: 1.859729
Agent: <VirtualWorker id:Agent_10 #objects:7> Train Epoch: 0 [3600/60000 (6%)]	Loss: 1.517509
Agent: <VirtualWorker id:Agent_10 #objects:7> Train Epoch: 0 [5400/60000 (9%)]	Loss: 1.164946
Agent: <VirtualWorker id:Agent_1 #objects:7> Train Epoch: 0 [7200/60000 (12%)]	Loss: 1.891937
Agent: <VirtualWorker id:Agent_1 #objects:7> Train Epoch: 0 [9000/60000 (15%)]	Loss: 1.696492
Agent: <VirtualWorker id:Agent_1 #objects:7> Train Epoch: 0 [10800/60000 (18%)]	Loss: 1.322264
Agent: <VirtualWorker id:Agent_2 #objects:7> Train Epoch: 0 [12600/60000 (21%)]	Loss: 2.005918
Agent: <VirtualWorker id:Agent_2 #objects:7> Train Epoch: 0 [14400/60000 (24%)]	Loss: 1.813337
Agent: <VirtualWorker id:Agent_2 #objects:7> Train Epoch: 0 [16200/60000 (27%)]	Loss: 1.406236
Agent: <VirtualWorker id:Agent_3 #objects:7> Train Epoch: 0

In [32]:
# Print the agent's deltas before boost.
delta_Agent_10['conv2.bias']

tensor([ 1.5115e-03,  3.7938e-03,  2.5545e-03,  2.2787e-03,  7.6362e-03,
         6.6458e-04, -4.1542e-04, -1.8627e-04,  5.1933e-03,  7.2931e-03,
         8.8161e-03, -3.1278e-03,  2.5482e-03,  4.3144e-05, -1.9124e-04,
        -5.5132e-04, -1.0824e-03,  7.6551e-03,  4.9327e-03,  6.9136e-03,
        -2.2229e-03,  5.7395e-03, -1.0221e-03,  8.6672e-04,  8.3970e-03,
         2.5776e-03,  3.8376e-03,  5.1200e-03,  2.3980e-03,  4.1267e-03,
         2.9473e-03, -1.6423e-04,  4.4161e-03, -4.5321e-03,  1.6304e-03,
        -9.4280e-04,  4.5454e-04, -2.4468e-04,  1.7016e-03,  3.8102e-05,
         4.4997e-03, -6.9509e-05, -6.1191e-04,  2.8482e-03,  1.0191e-02,
         1.8819e-04,  1.4196e-03, -1.7642e-04,  5.0120e-03, -3.8490e-03],
       device='cuda:0')

In [0]:
# boost malicious Agent 10's update
delta_Agent_10 = boost_update(delta_Agent_10,10.0, False)
agent_updates_ts = [delta_Agent_1, delta_Agent_2, delta_Agent_3, delta_Agent_4,delta_Agent_5, delta_Agent_6, delta_Agent_7, delta_Agent_8, delta_Agent_9, delta_Agent_10]

average_delta_all_Agents = avg_agent_updates(agent_updates_ts)

In [34]:
# Print the agent's deltas after boost.
delta_Agent_10['conv2.bias']

tensor([ 0.0151,  0.0379,  0.0255,  0.0228,  0.0764,  0.0066, -0.0042, -0.0019,
         0.0519,  0.0729,  0.0882, -0.0313,  0.0255,  0.0004, -0.0019, -0.0055,
        -0.0108,  0.0766,  0.0493,  0.0691, -0.0222,  0.0574, -0.0102,  0.0087,
         0.0840,  0.0258,  0.0384,  0.0512,  0.0240,  0.0413,  0.0295, -0.0016,
         0.0442, -0.0453,  0.0163, -0.0094,  0.0045, -0.0024,  0.0170,  0.0004,
         0.0450, -0.0007, -0.0061,  0.0285,  0.1019,  0.0019,  0.0142, -0.0018,
         0.0501, -0.0385], device='cuda:0')

In [35]:
mod_weights = updated_weights(Current_model,average_delta_all_Agents,False)
Current_model_ts_plus_n = Net().to(device)
Current_model_ts_plus_n.load_state_dict(mod_weights)
loss_, main_accuracy_ = test(args, Current_model_ts_plus_n, device, test_loader)
print('Timestamp {}. Loss is {:.6f}. Accuracy is {:.2f}'.format(1,loss_,main_accuracy_))

Timestamp 1. Loss is 1.887801. Accuracy is 59.01


In [0]:
data = mnist_full_train_dataset_bs[8][0].resize_((1,1,28,28)).cuda()
output = Current_model_ts_plus_n(data)
#test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
pred = output.argmax(1, keepdim=True)
pred

tensor([[5]], device='cuda:0')

In [0]:

model.load_state_dict(mod_weights)
Current_model.load_state_dict(model.state_dict())
#Current_model.load_state_dict(mod_weights)

# RERUN steps to get new deltas